In [14]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql
import calendar
import time
import json
import requests
from datetime import datetime
from threading import Timer

In [15]:
class DBUpdater:
    def __init__(self):
        """MariaDB DB 연결 및 구성"""
        self.conn = pymysql.connect(
            host='localhost', user='root', password='password', db='analyzer', charset='utf8')

        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                sector VARCHAR(50),
                last_update DATE,
                PRIMARY KEY (code)) ENGINE=InnoDB DEFAULT CHARSET=utf8;
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date)) ENGINE=InnoDB DEFAULT CHARSET=utf8;
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()

    def __del__(self):
        """MariaDB 연결 해제"""
        self.conn.close()

    def read_krx(self):
        """KRX 웹사이트 로부터 얻은 상장 법인 목록 파일을 읽어와서 DataFrame(company_info)으로 반환"""
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
        krx = pd.read_html(url, header=0)[0]
        krx = krx[['종목코드', '회사명', '업종']]
        krx = krx.rename(
            columns={'종목코드': 'code', '회사명': 'company', '업종': 'sector'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx

    def update_company_info(self):
        """company_info를 업데이트하고 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]

        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sector = krx.sector.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, sector, last_update) VALUES ('{code}', '{company}', '{sector}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company
                    time_now = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(
                        f"[{time_now}] #{idx+1:04d} REPLACE INTO company_info VALUES ({code}, {company}, {sector}, {today})")
                self.conn.commit()
                print('*')

    def read_naver(self, code, company, pages_to_fetch):
        """네이버 증권에서 읽어온 주식 시세를 DataFrame으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(
                url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split("=")
            lastpage = s[-1]
            df = pd.Dataframe()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                df = df.append(pd.read_html(requests.get(
                    pg_url, headers={"User-agent": 'Mozilla/5.0'}).text)[0])
                time_now = datetime.now().strftime('%Y-%m-%d %H:%M')
                print("[{}] {} ({}) : {:04d}/{:04d} pages are downloading......".format(
                    time_now, company, code, page, pages), end="\r")
            df = df.rename(columns={'날짜': 'date', '종가': 'close',
                           '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            df[['close', 'open', 'high', 'low', 'volume']] = df[[
                'close', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df

    def replace_df_into_db(self, df, num, code, company):
        """ DataFrame을 DB에 Replace"""
        with self.conn.cursor() as curs:
            for r in df.iterrtuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', '{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, {r.volume})"
                curs.execute(sql)
                
            self.conn.commit()
            print("[{}] #{:04d} {} ({}) : {} rows Replace Into Daily Price [✔]".format(datetime.now().strftime('%Y-%m-%d %H:%M'), num+1, company, code, len(df)))
                

    def update_daily_price(self, pages_to_fetch):
        """상장법인의 주식시세를 네이버에서 읽어서 업로드"""
        for idx, code in enumerate(self, codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_df_into_db(df, idx, code, self.codes[code])
        

    def execute_daily(self):
        self.update_company_info()
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 125
                config = {"pages_to_fetch": 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)
        
    if __name__ == '__main__':
        DBU = DBUpdater()
        DBU.execute_daily()
        

[2021-06-15 00:12] #0001 REPLACE INTO company_info VALUES (004840, DRB동일, 고무제품 제조업, 2021-06-15)
[2021-06-15 00:12] #0002 REPLACE INTO company_info VALUES (155660, DSR, 1차 비철금속 제조업, 2021-06-15)
[2021-06-15 00:12] #0003 REPLACE INTO company_info VALUES (001250, GS글로벌, 상품 종합 도매업, 2021-06-15)
[2021-06-15 00:12] #0004 REPLACE INTO company_info VALUES (294870, HDC현대산업개발, 건물 건설업, 2021-06-15)
[2021-06-15 00:12] #0005 REPLACE INTO company_info VALUES (092220, KEC, 반도체 제조업, 2021-06-15)
[2021-06-15 00:12] #0006 REPLACE INTO company_info VALUES (016380, KG동부제철, 1차 철강 제조업, 2021-06-15)
[2021-06-15 00:12] #0007 REPLACE INTO company_info VALUES (001390, KG케미칼, 기초 화학물질 제조업, 2021-06-15)
[2021-06-15 00:12] #0008 REPLACE INTO company_info VALUES (058860, KTis, 기타 정보 서비스업, 2021-06-15)
[2021-06-15 00:12] #0009 REPLACE INTO company_info VALUES (011070, LG이노텍, 전자부품 제조업, 2021-06-15)
[2021-06-15 00:12] #0010 REPLACE INTO company_info VALUES (066570, LG전자, 통신 및 방송 장비 제조업, 2021-06-15)
[2021-06-15 00:12] #0011 REP

[2021-06-15 00:12] #1123 REPLACE INTO company_info VALUES (032940, 원익, 기계장비 및 관련 물품 도매업, 2021-06-15)
[2021-06-15 00:12] #1124 REPLACE INTO company_info VALUES (030530, 원익홀딩스, 특수 목적용 기계 제조업, 2021-06-15)
[2021-06-15 00:12] #1125 REPLACE INTO company_info VALUES (101730, 위메이드맥스, 소프트웨어 개발 및 공급업, 2021-06-15)
[2021-06-15 00:12] #1126 REPLACE INTO company_info VALUES (024800, 유성티엔에스, 육상 여객 운송업, 2021-06-15)
[2021-06-15 00:12] #1127 REPLACE INTO company_info VALUES (263770, 유에스티, 1차 철강 제조업, 2021-06-15)
[2021-06-15 00:12] #1128 REPLACE INTO company_info VALUES (065440, 이루온, 소프트웨어 개발 및 공급업, 2021-06-15)
[2021-06-15 00:12] #1129 REPLACE INTO company_info VALUES (131400, 이브이첨단소재, 전자부품 제조업, 2021-06-15)
[2021-06-15 00:12] #1130 REPLACE INTO company_info VALUES (102710, 이엔에프테크놀로지, 기타 화학제품 제조업, 2021-06-15)
[2021-06-15 00:12] #1131 REPLACE INTO company_info VALUES (063760, 이엘피, 측정, 시험, 항해, 제어 및 기타 정밀기기 제조업; 광학기기 제외, 2021-06-15)
[2021-06-15 00:12] #1132 REPLACE INTO company_info VALUES (083470, 이엠앤아이, 플라스

[2021-06-15 00:12] #2273 REPLACE INTO company_info VALUES (108490, 로보티즈, 특수 목적용 기계 제조업, 2021-06-15)
[2021-06-15 00:12] #2274 REPLACE INTO company_info VALUES (071280, 로체시스템즈, 특수 목적용 기계 제조업, 2021-06-15)
[2021-06-15 00:12] #2275 REPLACE INTO company_info VALUES (058470, 리노공업, 전자부품 제조업, 2021-06-15)
[2021-06-15 00:12] #2276 REPLACE INTO company_info VALUES (305090, 마이크로디지탈, 의료용 기기 제조업, 2021-06-15)
[2021-06-15 00:12] #2277 REPLACE INTO company_info VALUES (220630, 맘스터치, 음·식료품 및 담배 도매업, 2021-06-15)
[2021-06-15 00:12] #2278 REPLACE INTO company_info VALUES (093520, 매커스, 기계장비 및 관련 물품 도매업, 2021-06-15)
[2021-06-15 00:12] #2279 REPLACE INTO company_info VALUES (041920, 메디아나, 의료용 기기 제조업, 2021-06-15)
[2021-06-15 00:12] #2280 REPLACE INTO company_info VALUES (218150, 미래생명자원, 동물용 사료 및 조제식품 제조업, 2021-06-15)
[2021-06-15 00:12] #2281 REPLACE INTO company_info VALUES (064520, 바른전자, 반도체 제조업, 2021-06-15)
[2021-06-15 00:12] #2282 REPLACE INTO company_info VALUES (267790, 배럴, 봉제의복 제조업, 2021-06-15)
[2021-06-1

[2021-06-15 00:12] #0001 동화약품 (000020) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0002 KR모터스 (000040) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0003 경방 (000050) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0004 메리츠화재 (000060) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0005 삼양홀딩스 (000070) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0006 하이트진로 (000080) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0007 유한양행 (000100) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0008 CJ대한통운 (000120) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0009 하이트진로홀딩스 (000140) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0010 두산 (000150) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0011 성창기업지주 (000180) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0012 DL (000210) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #00

[2021-06-15 00:12] #0102 도화엔지니어링 (002150) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0103 삼양통상 (002170) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0104 한국수출포장공업 (002200) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0105 동성제약 (002210) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0106 한일철강 (002220) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0107 피에스텍 (002230) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0108 고려제강 (002240) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0109 롯데푸드 (002270) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0110 삼일기업공사 (002290) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0111 아세아제지 (002310) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0112 한진 (002320) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] #0113 넥센타이어 (002350) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:12] 

...

[2021-06-15 00:17] #2035 코스메카코리아 (241710) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2036 메카로 (241770) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2037 오션브릿지 (241790) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2038 피씨엘 (241820) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2039 에이스토리 (241840) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2040 나무기술 (242040) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2041 피엔아이컴퍼니 (242350) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2042 휴온스 (243070) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2043 신흥에스이씨 (243840) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2044 굿센 (243870) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2045 올리패스 (244460) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #2046 나눔테크 (244880) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:17] #204

[2021-06-15 00:18] #2136 지엔원에너지 (270520) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2137 에브리봇 (270660) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2138 뉴트리 (270870) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2139 오리온 (271560) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2140 다이오진 (271850) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2141 제일약품 (271980) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2142 케이엔제이 (272110) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2143 한화시스템 (272210) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2144 이녹스첨단소재 (272290) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2145 진에어 (272450) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2146 삼양패키징 (272550) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2147 와이즈버즈 (273060) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #21

[2021-06-15 00:18] #2237 피플바이오 (304840) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2238 마이크로디지탈 (305090) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2239 에스제이그룹 (306040) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2240 세아제강 (306200) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2241 네온테크 (306620) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2242 SK4호스팩 (307070) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2243 하나머스트제6호스팩 (307160) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2244 아이엘사이언스 (307180) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2245 원바이오젠 (307280) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2246 국전약품 (307750) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2247 상상인이안1호스팩 (307870) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2248 컴퍼니케이 (307930) : 10 rows > REPLACE INTO daily_price [OK]
[202

[2021-06-15 00:18] #2336 SK5호스팩 (337450) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2337 유엑스엔 (337840) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2338 브랜드엑스코퍼레이션 (337930) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2339 NH프라임리츠 (338100) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2340 뷰노 (338220) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2341 교촌에프앤비 (339770) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2342 아이비김영 (339950) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2343 하이제5호스팩 (340120) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2344 SK6호스팩 (340350) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2345 유안타제6호스팩 (340360) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2346 한화플러스제1호스팩 (340440) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2347 티앤엘 (340570) : 10 rows > REPLACE INTO daily_price [OK]
[2

[2021-06-15 00:18] #2437 컬러레이 (900310) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2438 윙입푸드 (900340) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2439 SBI핀테크솔루션즈 (950110) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2440 엑세스바이오 (950130) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2441 잉글우드랩 (950140) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2442 코오롱티슈진 (950160) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2443 JTC (950170) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2444 SNK (950180) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2445 미투젠 (950190) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2446 소마젠 (950200) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2447 프레스티지바이오파마 (950210) : 10 rows > REPLACE INTO daily_price [OK]
[2021-06-15 00:18] #2448 네오이뮨텍 (950220) : 10 rows > REPLACE INTO daily_price [OK]
Waiting for ne